Testing delta probability for weeklies and monthlies\n",
    - Does delta match probability of occurrence\n",
    - Check Kelly criterion for long straddles/delta neutral back ratios\n",
        - To calculate expected losses and gains, use options pricing module to interpolate prices at given percentage\n",
        moves and use the normal pdf as the probability weightings\n
Check earnings returns post announcement factors\n",
    - Factors:\n",
        - Number of times beaten earnings (Dummy Variable)\n",
        - Consecutive earnings beats\n",
        - Consecutive earnings upsets\n",
        - 3 Month Trend before earnings\n",
        - YTD Trend\n",
        - Momentum of monthly returns (20 day, 60 day)\n"
        
        
price-to-book (P / B), price-to-earnings (P / E), price-to-free cashflow (P / FCF) and enterprise value-to-EBITDA (EV / EBITDA

In [2]:
import datetime as dt

import numpy as np
import pandas as pd
from pandas_datareader.data import Options
from py_vollib.black_scholes_merton.implied_volatility import *
# ts = TimeSeries(key='5HZEUI5AFJB06BUK',output_format='pandas')
import py_vollib
from py_vollib.black_scholes_merton.implied_volatility import *
from py_vollib.black_scholes_merton.greeks.analytical import *
import plotly
import os
import pandas_market_calendars as mcal
import json
from pandas.io.json import json_normalize
import urllib.request as req
import time
nyse = mcal.get_calendar('NYSE')
from helpers import *
from tickers import tickers

from scipy.stats import norm as norm
from pandas import ExcelWriter
from pandas import ExcelFile
from openpyxl import load_workbook
from yahoo_query import *

In [11]:
# '''
# functions list:

#     maturities(dt.datetime()) --> [float(front_wgt), float(back_wgt)]

#     optionslam_scrape(str[ticker]) --> dict[earnings]

#     yahoo_table_parse(str[raw_html_table]) --> DataFrame[ticker]

#     yahoo_earnings(dt.datetime()) --> DataFrame[earnings_on_date]

#     fundamentals(str[ticker]) --> DataFrame[stock_fundamentals]

#     get_fundas(list[ticker_lst]) --> DataFrame[stock_fundamentals]

#     historical_data(str[ticker], int[day_number], int[rolling_window], outsize[str]) --> DataFrame[daily_stock_data]

#     current_volatility(list[ticker_lst], int[roll]) --> DataFrame[stock_volatilities]

#     all_options(str[ticker], bool[greeks]) --> DataFrame[options_chains]

#     earnings_condor(str[ticker], int[max_gap], int[dte_thresh], float[|money_thresh| <= 1]) -- DataFrame[condors], DataFrame[puts], DataFrame[calls]

#     write_excel(str[filename], list[str[sheetnames]], list[dataframes]) --> void()

#     curr_stock_data(str[ticker]) --> DataFrame[stock_info]

#     curr_batch_quotes(list_of_string[tickers]) --> DataFrame[stock_info]

#     past_earnings(str[ticker]) --> DataFrame[earnings_info]

#     earnings_history(str[ticker]) --> [DataFrame[earnings_estimate], DataFrame[past_earnings], DataFrame[earnings_estimate_change]]

#     av_data(str[ticker]) --> DataFrame[ticker_open, ticker_close]

#     av_batch(list_of_str[tickers]) --> DataFrame[tickers_closes]

#     check_mkt_corr(int[corr_rolling_window],int[plot_window]) --> DataFrame[rolling_corr]

#     vvix_check() --> DataFrame[VVIX Data]

#     earnings_longs(list_of_str[ticker], float[bid_ask_spread]) --> DataFrame[option_chains]

#     all_options_v2(str[ticker], int[dte_ub], int[dte_lb], float[moneyness]) --> DataFrame[option_chains]

#     yahoo_options_query(str[ticker], int[dte_ub], int[dte_lb]) --> DataFrame[option_chains]

#     greek_calc(DataFrame[option_chain], str[prem_price_use], str[day_format], float[interest_rate], float[dividend_rate])

#     price_sim(DataFrame[options_df], float[price_change], float[vol_change], int[days_change], str[output = 'All'],
#               str[skew = 'flat'], str[day_format = 'trading'], float[interest_rate = 0.0193], float[dividend_rate = 0],
#               float[prem_price_use = 'Mid'])


#     position_sim(DataFrame[position_df], list_of_int[holdings], int[shares],
#                  float[price_change], float[vol_change], int[dte_change], str[output = 'All'],
#                  str[skew = 'flat'], str[prem_price_use = 'Mid'], str[day_format = 'trading'], 
#                  float[interest_rate = 0.0193], float[dividend_rate = 0])

#     yahoo_fundamentals(list_of_str[tickers]) --> DataFrame[fundamentals]

# '''

# stock_list = pd.read_csv('optionablestocks.csv')['OPTION SYMBOL'].tolist()
# stock_list = list(set(tickers[3:] + stock_list))

cad_names = pd.read_csv('cad_names.csv')['Symbol'].tolist()
us_names = pd.read_csv('us_names.csv')['Symbol'].tolist()
start_time = time.time()

def pull_data(ticker):
    yahoo_data = yahoo_query(ticker,dt.datetime(2018,1,1))
    yahoo_data.full_info_query()
    earnings_info_quarter = yahoo_data.earnings_history.join(yahoo_data.cashFlowStatementQuarter).join(yahoo_data.incomeStatementQuarter.drop(['netIncome','maxAge'],
                                                                                                                                      axis = 1),
                                                                                               rsuffix='_income').join(yahoo_data.balanceSheetQuarter,
                                                                                                                       rsuffix = '_balance')
    
    annual_info = yahoo_data.cashFlowStatementAnnual.join(yahoo_data.incomeStatementAnnual.drop(['netIncome','maxAge'],
                                                                                                  axis = 1),
                                                           rsuffix='_income').join(yahoo_data.balanceSheetAnnual,
                                                                                   rsuffix = '_balance')
#     earnings_info['earnBeatsBefore'] = 0
#     earnings_info['earnMissBefore'] = 0

#     for idx, row in earnings_info.iterrows():
#         earnings_info.loc[idx,'earnBeatsBefore'] = len(earnings_info[(earnings_info.index <= idx) & (earnings_info.epsDifference > 0)])
#         earnings_info.loc[idx,'earnMissBefore'] = len(earnings_info[(earnings_info.index <= idx) & (earnings_info.epsDifference <= 0)])
#     earnings_info = earnings_info.shift(1)


#     earnings_moves = past_earnings(ticker).sort_index()
#     earnings_moves = earnings_moves[(earnings_moves.index > min(yahoo_data.earnings_history.index) - dt.timedelta(days = 92)) &
#                                     (earnings_moves.index <= max(yahoo_data.earnings_history.index))].sort_index()


#     earnings_df = pd.concat([earnings_info.reset_index(), 
#                              earnings_moves.reset_index()], axis = 1)
    earnings_info_quarter.columns = ['quarter' if col == 'index' else col for col in earnings_info_quarter.columns.tolist()]
    earnings_info_quarter['Underlying'] = ticker
    annual_info.columns = ['year' if col == 'index' else col for col in annual_info.columns.tolist()]
    annual_info['Underlying'] = ticker

    ### separate df for current key measures
    keyMetrics = yahoo_data.profile.join(yahoo_data.keyStats).join(yahoo_data.finData, rsuffix = '_finData')

    return (earnings_info_quarter, annual_info, keyMetrics)

def download_yahoo_data(ticker_list, retries = 10):

    earnings_lst = []
    annual_lst = []
    keyStats_lst = []

    item_counter = 0
    total_length = len(ticker_list)
    failed_list = []

    for ticker in ticker_list:
        try:
            curr_earnings, curr_annual_info, curr_keyStats = pull_data(ticker)
            earnings_lst.append(curr_earnings)
            annual_lst.append(curr_annual_info)
            keyStats_lst.append(curr_keyStats)
            print('Accepted: {}'.format(ticker))
        except:
            for i in range(retries):
                try:
                    curr_earnings, curr_annual_info, curr_keyStats = pull_data(ticker)
                    earnings_lst.append(curr_earnings)
                    annual_lst.append(curr_annual_info)
                    keyStats_lst.append(curr_keyStats)
                    print('Accepted: {}'.format(ticker))
                except:
                    continue
            print('Failed: {}'.format(ticker))
            failed_list.append(ticker)

        item_counter += 1
        print('{0:.2f}% Completed'.format(item_counter/total_length*100))
        print('{} total failures'.format(len(failed_list)))

    earnings_df = pd.concat(earnings_lst, axis = 0)
    annual_df = pd.concat(annual_lst, axis = 0)
    #earnings_df = earnings_df.reset_index()[earnings_df.columns]
    keyStats_df = pd.concat(keyStats_lst, axis = 0)

    return earnings_df, annual_df, keyStats_df, failed_list

def fin_ratios(earnings_df):

    ratios_df = earnings_df[['Underlying','timestamp','quarter','1Year', '1month', '3month', '6month', 
                             'PostEarningsReturn','industry', 'sector']]

In [21]:
earnings_df, annual_df, keyStats_df, failed_list = download_yahoo_data(us_names, retries = 10)

datenow = dt.datetime.today().strftime("%Y-%m-%d")
earnings_df.to_csv('us_quarterly_{}.csv'.format(datenow))
annual_df.to_csv('us_annual_{}.csv'.format(datenow))
keyStats_df.to_csv('us_keystats_{}.csv'.format(datenow))


Failed: AAAU
0.01% Completed
1 total failures
Failed: AADR
0.02% Completed
2 total failures
Failed: AAMC
0.03% Completed
3 total failures
Failed: AAU
0.05% Completed
4 total failures
Failed: ACES
0.06% Completed
5 total failures
Failed: ACIM
0.07% Completed
6 total failures
Failed: ACSI
0.08% Completed
7 total failures
Accepted: ACU
0.09% Completed
7 total failures
Failed: ACWF
0.10% Completed
8 total failures
Failed: ACWV
0.12% Completed
9 total failures
Accepted: ACY
0.13% Completed
9 total failures
Failed: AE
0.14% Completed
10 total failures
Failed: AEF
0.15% Completed
11 total failures
Failed: AFK
0.16% Completed
12 total failures
Failed: AFTY
0.17% Completed
13 total failures
Failed: AGF
0.18% Completed
14 total failures
Failed: AGG
0.20% Completed
15 total failures
Failed: AGGE
0.21% Completed
16 total failures
Failed: AGGP
0.22% Completed
17 total failures
Failed: AGGY
0.23% Completed
18 total failures
Failed: AGQ
0.24% Completed
19 total failures
Failed: AGT
0.25% Completed
20

Failed: BWZ
2.02% Completed
156 total failures
Failed: BYLD
2.03% Completed
157 total failures
Failed: BZF
2.04% Completed
158 total failures
Failed: BZQ
2.05% Completed
159 total failures
Failed: CALF
2.07% Completed
160 total failures
Failed: CANE
2.08% Completed
161 total failures
Accepted: CANF
2.09% Completed
161 total failures
Failed: CAPE
2.10% Completed
162 total failures
Failed: CAW
2.11% Completed
163 total failures
Failed: CBND
2.12% Completed
164 total failures
Failed: CBON
2.14% Completed
165 total failures
Failed: CCA
2.15% Completed
166 total failures
Failed: CCF
2.16% Completed
167 total failures
Failed: CCOR
2.17% Completed
168 total failures
Accepted: CDOR
2.18% Completed
168 total failures
Failed: CEF
2.19% Completed
169 total failures
Failed: CEFL
2.20% Completed
170 total failures
Failed: CEFS
2.22% Completed
171 total failures
Failed: CEI
2.23% Completed
172 total failures
Failed: CEMB
2.24% Completed
173 total failures
Failed: CET
2.25% Completed
174 total failur

Failed: DMRI
4.01% Completed
317 total failures
Failed: DMRL
4.02% Completed
318 total failures
Failed: DMRM
4.03% Completed
319 total failures
Failed: DMRS
4.04% Completed
320 total failures
Failed: DNL
4.05% Completed
321 total failures
Accepted: DNN
4.06% Completed
321 total failures
Failed: DNO
4.07% Completed
322 total failures
Failed: DOD
4.09% Completed
323 total failures
Failed: DOG
4.10% Completed
324 total failures
Failed: DOGS
4.11% Completed
325 total failures
Failed: DOL
4.12% Completed
326 total failures
Failed: DON
4.13% Completed
327 total failures
Failed: DOO
4.14% Completed
328 total failures
Failed: DPK
4.16% Completed
329 total failures
Failed: DPST
4.17% Completed
330 total failures
Failed: DPW
4.18% Completed
331 total failures
Failed: DRIP
4.19% Completed
332 total failures
Failed: DRN
4.20% Completed
333 total failures
Failed: DRR
4.21% Completed
334 total failures
Failed: DRSK
4.22% Completed
335 total failures
Failed: DRV
4.24% Completed
336 total failures
Fai

Failed: EUSA
6.00% Completed
483 total failures
Failed: EUSC
6.01% Completed
484 total failures
Failed: EUXL
6.02% Completed
485 total failures
Accepted: EVBN
6.04% Completed
485 total failures
Failed: EVI
6.05% Completed
486 total failures
Failed: EVIX
6.06% Completed
487 total failures
Failed: EVJ
6.07% Completed
488 total failures
Failed: EVM
6.08% Completed
489 total failures
Failed: EVO
6.09% Completed
490 total failures
Failed: EVP
6.11% Completed
491 total failures
Failed: EVV
6.12% Completed
492 total failures
Failed: EVX
6.13% Completed
493 total failures
Failed: EVY
6.14% Completed
494 total failures
Failed: EWA
6.15% Completed
495 total failures
Failed: EWC
6.16% Completed
496 total failures
Failed: EWD
6.17% Completed
497 total failures
Failed: EWEM
6.19% Completed
498 total failures
Failed: EWG
6.20% Completed
499 total failures
Failed: EWGS
6.21% Completed
500 total failures
Failed: EWH
6.22% Completed
501 total failures
Failed: EWI
6.23% Completed
502 total failures
Fail

Failed: FRLG
7.99% Completed
654 total failures
Failed: FRN
8.00% Completed
655 total failures
Accepted: FSI
8.01% Completed
655 total failures
Accepted: FSP
8.02% Completed
655 total failures
Failed: FSTA
8.03% Completed
656 total failures
Failed: FTEC
8.04% Completed
657 total failures
Failed: FTF
8.06% Completed
658 total failures
Failed: FTLS
8.07% Completed
659 total failures
Failed: FTNW
8.08% Completed
660 total failures
Failed: FTSD
8.09% Completed
661 total failures
Failed: FTVA
8.10% Completed
662 total failures
Failed: FUD
8.11% Completed
663 total failures
Failed: FUE
8.13% Completed
664 total failures
Failed: FUT
8.14% Completed
665 total failures
Failed: FUTY
8.15% Completed
666 total failures
Failed: FVAL
8.16% Completed
667 total failures
Failed: FVD
8.17% Completed
668 total failures
Failed: FVL
8.18% Completed
669 total failures
Failed: FWDB
8.19% Completed
670 total failures
Failed: FWDD
8.21% Completed
671 total failures
Failed: FWDI
8.22% Completed
672 total failur

...

Failed: RWGE
92.26% Completed
4640 total failures
Accepted: RWT
92.27% Completed
4640 total failures
Accepted: RXN
92.28% Completed
4640 total failures
Failed: RXN-A
92.29% Completed
4641 total failures
Accepted: RY
92.30% Completed
4641 total failures
Failed: RY-T
92.31% Completed
4642 total failures
Failed: RYA-A
92.32% Completed
4643 total failures
Accepted: RYAM
92.34% Completed
4643 total failures
Failed: RYB
92.35% Completed
4644 total failures
Accepted: RYI
92.36% Completed
4644 total failures
Accepted: RYN
92.37% Completed
4644 total failures
Failed: RZA
92.38% Completed
4645 total failures
Failed: RZB
92.39% Completed
4646 total failures
Accepted: S
92.41% Completed
4646 total failures
Failed: SA
92.42% Completed
4647 total failures
Failed: SAB
92.43% Completed
4648 total failures
Failed: SAF
92.44% Completed
4649 total failures
Accepted: SAFE
92.45% Completed
4649 total failures
Accepted: SAH
92.46% Completed
4649 total failures
Accepted: SAIC
92.47% Completed
4649 total fail

Accepted: SRC
94.13% Completed
4708 total failures
Failed: SRC-A
94.14% Completed
4709 total failures
Accepted: SRE
94.15% Completed
4709 total failures
Failed: SRE-A
94.16% Completed
4710 total failures
Failed: SRE-B
94.17% Completed
4711 total failures
Failed: SRF
94.18% Completed
4712 total failures
Failed: SRG
94.19% Completed
4713 total failures
Failed: SRG-A
94.21% Completed
4714 total failures
Accepted: SRI
94.22% Completed
4714 total failures
Accepted: SRLP
94.23% Completed
4714 total failures
Accepted: SRT
94.24% Completed
4714 total failures
Failed: SRV
94.25% Completed
4715 total failures
Accepted: SSD
94.26% Completed
4715 total failures
Accepted: SSI
94.28% Completed
4715 total failures
Failed: SSL
94.29% Completed
4716 total failures
Accepted: SSTK
94.30% Completed
4716 total failures
Accepted: SSW
94.31% Completed
4716 total failures
Failed: SSW-D
94.32% Completed
4717 total failures
Failed: SSW-E
94.33% Completed
4718 total failures
Failed: SSW-G
94.34% Completed
4719 t

Accepted: TMST
95.99% Completed
4776 total failures
Accepted: TNC
96.01% Completed
4776 total failures
Accepted: TNET
96.02% Completed
4776 total failures
Failed: TNK
96.03% Completed
4777 total failures
Accepted: TNP
96.04% Completed
4777 total failures
Failed: TNP-B
96.05% Completed
4778 total failures
Failed: TNP-C
96.06% Completed
4779 total failures
Failed: TNP-D
96.08% Completed
4780 total failures
Failed: TNP-E
96.09% Completed
4781 total failures
Failed: TNP-F
96.10% Completed
4782 total failures
Accepted: TOL
96.11% Completed
4782 total failures
Accepted: TOO
96.12% Completed
4782 total failures
Failed: TOO-A
96.13% Completed
4783 total failures
Failed: TOO-B
96.14% Completed
4784 total failures
Failed: TOO-E
96.16% Completed
4785 total failures
Accepted: TOT
96.17% Completed
4785 total failures
Accepted: TOWR
96.18% Completed
4785 total failures
Accepted: TPB
96.19% Completed
4785 total failures
Accepted: TPC
96.20% Completed
4785 total failures
Failed: TPG.H
96.21% Completed

Accepted: VIPS
97.86% Completed
4841 total failures
Accepted: VIV
97.88% Completed
4841 total failures
Accepted: VJET
97.89% Completed
4841 total failures
Failed: VKQ
97.90% Completed
4842 total failures
Accepted: VLO
97.91% Completed
4842 total failures
Accepted: VLP
97.92% Completed
4842 total failures
Accepted: VLRS
97.93% Completed
4842 total failures
Failed: VLT
97.95% Completed
4843 total failures
Accepted: VLY
97.96% Completed
4843 total failures
Failed: VLY-A
97.97% Completed
4844 total failures
Failed: VLY-B
97.98% Completed
4845 total failures
Failed: VLY.W
97.99% Completed
4846 total failures
Accepted: VMC
98.00% Completed
4846 total failures
Accepted: VMI
98.01% Completed
4846 total failures
Failed: VMO
98.03% Completed
4847 total failures
Accepted: VMW
98.04% Completed
4847 total failures
Failed: VNCE
98.05% Completed
4848 total failures
Failed: VNE
98.06% Completed
4849 total failures
Accepted: VNO
98.07% Completed
4849 total failures
Failed: VNO-K
98.08% Completed
4850 t

Accepted: XRX
99.73% Completed
4901 total failures
Accepted: XYL
99.75% Completed
4901 total failures
Accepted: Y
99.76% Completed
4901 total failures
Accepted: YELP
99.77% Completed
4901 total failures
Accepted: YEXT
99.78% Completed
4901 total failures
Accepted: YPF
99.79% Completed
4901 total failures
Accepted: YRD
99.80% Completed
4901 total failures
Accepted: YUM
99.82% Completed
4901 total failures
Accepted: YUMC
99.83% Completed
4901 total failures
Accepted: ZAYO
99.84% Completed
4901 total failures
Failed: ZB-A
99.85% Completed
4902 total failures
Failed: ZB-G
99.86% Completed
4903 total failures
Failed: ZB-H
99.87% Completed
4904 total failures
Accepted: ZBH
99.88% Completed
4904 total failures
Failed: ZBK
99.90% Completed
4905 total failures
Accepted: ZEN
99.91% Completed
4905 total failures
Failed: ZF
99.92% Completed
4906 total failures
Failed: ZNH
99.93% Completed
4907 total failures
Accepted: ZOES
99.94% Completed
4907 total failures
Accepted: ZTO
99.95% Completed
4907 to

In [50]:
# earnings, ks, failed_list = download_yahoo_data(stock_list, retries = 10)
# earnings = earnings.drop(earnings.isnull().sum().sort_values().tail(13).index.tolist(), axis = 1)
# earnings.to_csv('full_data.csv')

fs_df = pd.read_csv('full_data.csv', index_col = 0)

fs = fs_df[['Underlying']]
fs['curr_ratio'] = fs_df['totalCurrentAssets']/fs_df['totalCurrentLiabilities']
fs['totaldebtequity'] = fs_df['totalLiab']/fs_df['totalStockholderEquity']
fs['longdebtequity'] = fs_df['longTermDebt']/fs_df['totalStockholderEquity']
fs['shortdebtequity'] = fs_df['shortLongTermDebt']/fs_df['totalStockholderEquity']
fs['daySalesOut'] = (fs_df['netReceivables']/fs_df['totalRevenue'].replace(0,np.nan))*365
fs['intangToBook'] = fs_df['intangibleAssets']/fs_df['totalStockholderEquity']
fs['invToRev'] = fs_df['inventory']/fs_df['totalRevenue'].replace(0,np.nan)
fs['ltdebtToCap'] = fs_df['longTermDebt']/fs_df['investments']
fs['stdebtToCap'] = fs_df['shortLongTermDebt']/fs_df['investments']
fs['ltdebtPercent'] = fs_df['longTermDebt']/fs_df['totalLiab']
fs['stdebtPercent'] = fs_df['shortLongTermDebt']/fs_df['totalLiab']

fs['grossMargin'] = fs_df['grossProfit']/fs_df['totalRevenue'].replace(0,np.nan)
fs['profitMargin'] = fs_df['netIncome']/fs_df['totalRevenue'].replace(0,np.nan)
fs['operatingMargin'] = fs_df['operatingIncome']/fs_df['totalRevenue'].replace(0,np.nan)

fs['operatingCashflow'] = fs_df['totalCashFromOperatingActivities']/fs_df['totalRevenue'].replace(0,np.nan)
fs['assetEfficiency'] = fs_df['totalCashFromOperatingActivities']/fs_df['totalAssets']
fs['currLiabCoverage'] = fs_df['totalCashFromOperatingActivities']/fs_df['totalCurrentLiabilities']
fs['ltDebtCoverage'] = fs_df['totalCashFromOperatingActivities']/fs_df['longTermDebt']
fs['cashGeneratingPower'] = fs_df['totalCashFromOperatingActivities']/(fs_df['totalCashFromOperatingActivities'] + fs_df['totalCashFromFinancingActivities'] + fs_df['totalCashflowsFromInvestingActivities'])

fs_pct = []
for name in fs.Underlying.drop_duplicates().tolist():
    pct = fs[fs['Underlying'] == name].sort_index()
    pct[pct.columns.tolist()[1:]] = pct[pct.columns.tolist()[1:]].pct_change()
    fs_pct.append(pct.tail(len(pct)-1))

fs_pct = pd.concat(fs_pct, axis = 0)

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

In [55]:
filename = 'Leaps Filtering.xlsx'
ks = pd.read_excel(filename, sheetname = 'keystats', index_col = 0).drop_duplicates()

book = load_workbook(filename)
writer = pd.ExcelWriter(filename, engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

fs_pct.drop_duplicates().to_excel(writer, "finpctdata")
fs.drop_duplicates().to_excel(writer,"findata")
ks.drop('industry',axis = 1).groupby('sector').mean().to_excel(writer, "keystatsAvg")
writer.save()

## Checking Key Stats on Single Names
### For each industry filter names that have:
1. positive earnings growth
2. positive profit margin
3. lower than industry average p/b, p/e, ev/ebitda, debt/equity
4. rising margins quarter over quarter
5. decreasing debt/equity quarter over quarter
6. decreasing days of sales outstanding
7. lower than industry average inventory/revenue ratio

In [146]:
filename = 'Leaps Filtering.xlsx'
ks = pd.read_excel(filename, sheetname = 'keystats', index_col = 0).drop_duplicates()
ksavg = pd.read_excel(filename, sheetname = 'keystatsAvg', index_col = 0).drop_duplicates()
finpct = pd.read_excel(filename, sheetname = 'finpctdata', index_col = 0).drop_duplicates().fillna(0)
fin = pd.read_excel(filename, sheetname = 'findata', index_col = 0).drop_duplicates()

# Creating scores of pct changes
tickers = finpct.Underlying.drop_duplicates().tolist()
columns = finpct.columns.tolist()[1:]
values = ['inc','dec','dec','dec','dec','flat','dec','dec','dec',
          'dec','dec','inc','inc','inc','inc','inc','inc','inc',
          'inc']
field_directions = dict(zip(columns,values))
scores = pd.DataFrame(index = tickers, columns = columns)

for name in tickers:
    for field in columns:
        curr_fin = finpct[(finpct['Underlying'] == name)][field]
        if len(curr_fin) > 1:
            if field_directions[field] == 'inc':
                point = len(curr_fin) == sum(curr_fin >= 0)
            elif field_directions[field] == 'dec':
                point = len(curr_fin) == sum(curr_fin <= 0)
            else:
                point = False
        else:
            point = False
            
        scores.loc[name,field] = point*1
        
scores['Score'] = scores.sum(axis = 1)

In [153]:
top_names = scores[scores['Score'] >= 10].sort_values('Score', ascending = False)
ks_col_order = ['currentPrice', 'sector', 'industry',
                '52WeekChange', 'SandP52WeekChange', 'beta', 'bookValue',
                'currentRatio', 'debtToEquity', 'earningsGrowth', 'revenueGrowth', 
                'returnOnAssets', 'returnOnEquity',
                'earningsQuarterlyGrowth', 'heldPercentInsiders', 'heldPercentInstitutions', 
                'sharesPercentSharesOut', 'shortPercentOfFloat', 
                'ebitda', 'ebitdaMargins',
                'enterpriseToEbitda', 'enterpriseToRevenue', 'enterpriseValue',
                'floatShares', 'forwardEps', 'forwardPE', 'freeCashflow',
                'fullTimeEmployees', 'grossMargins', 'grossProfits',
                'netIncomeToCommon', 'operatingCashflow', 'profitMargins','operatingMargins',
                'overallRisk', 'pegRatio', 'priceToBook', 
                'quickRatio', 'recommendationKey',
                'recommendationMean', 
                'revenuePerShare', 'sharesOutstanding',
                'sharesShort', 'sharesShortPriorMonth',
                'shortRatio', 'targetHighPrice',
                'targetLowPrice', 'targetMeanPrice', 'targetMedianPrice', 'totalCash',
                'totalCashPerShare', 'totalDebt', 'totalRevenue', 'trailingEps']

In [154]:
filteredfilename = 'filtered leaps.xlsx'
top_names.to_excel(filteredfilename,'scores')

book = load_workbook(filteredfilename)
writer = pd.ExcelWriter(filteredfilename, engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

finpct[finpct.Underlying.isin(top_names.index.tolist())].to_excel(writer, 'finpctdata')
fin[fin.Underlying.isin(top_names.index.tolist())].to_excel(writer, 'findata')
ks[ks_col_order][ks.index.isin(top_names.index.tolist())].to_excel(writer,'keystats')
ks.drop('industry',axis = 1).groupby('sector').mean().to_excel(writer, "keystatsAvg")

writer.save()



In [173]:
ftnt = yahoo_query('BMO.TO', dt.datetime(2018,1,1))
ftnt.full_info_query()

In [175]:
ftnt.executives

,age,exercisedValue,fiscalYear,name,totalPay,unexercisedValue
title,,,,,,
CEO & Director,45,0,2016,Mr. William Darryl White,3286320,0
Chief Financial Officer,NaN,0,2016,Mr. Thomas E. Flynn,1404960,0
Chief Technology & Operations Officer,NaN,0,2016,Mr. Jean-Michel R. Arès,2538160,0
Group Head of BMO Capital Markets,NaN,0,NaN,Mr. Patrick Cronin,NaN,0
Exec. VP & Chief Information Officer,NaN,0,NaN,Mr. Steve Tennyson,NaN,0


In [176]:
ftnt.purchaseActivity

,buyInfoCount,buyInfoShares,maxAge,netInfoCount,netInfoShares,period,sellInfoCount,totalInsiderShares
BMO.TO,0,0,1,0,0,6m,0,0
